<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# FEC - Création du modèle de données

**Tags:** #fec #datamodel #finance #snippet #operation

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** Ce notebook permet de lire la base de données FEC et de le transformer dans le format du modèle de données attendu pour générer des états financiers.

## Input

### Import libraries

In [1]:
import pandas as pd
import glob
import re
import os
import naas
from datetime import datetime

### Setup Variables

In [2]:
# Inputs
input_folder_path = "/home/ftp/FEC-engine/outputs/FEC/BDD_INIT"
to_map = {
    "EcritureDate": "DATE",
    "CompteNum": "COMPTE_NUM",
    "CompteLib": "COMPTE_LIB",
    "EcritureLib": "ECRITURE_LIB",
    "Debit": "DEBIT",
    "Credit": "CREDIT",
}

# Outputs
output_folder_path = "/home/ftp/FEC-engine/outputs/FEC/BDD_FEC"

## Model

### Récupération du dernier fichier input

In [3]:
def read_last_csv(folder_path):
    # Init
    df = pd.DataFrame()
    
    # List files in folder
    files = sorted(glob.glob(f"{folder_path}/*.csv"), reverse=True)
    
    # Read last csv files
    if len(files) > 0:
        last_file = files[-1]
        print("📁 Last file:", last_file)
        df = pd.read_csv(last_file, sep=";", decimal=",")
    return df

df_input = read_last_csv(input_folder_path)
print("✅ Row fetched:", len(df_input))
df_input.head(1)

📁 Last file: /home/ftp/FEC-engine/outputs/FEC/BDD_INIT/20230524122731_FEC_BDD_INIT.csv
✅ Row fetched: 4532


,JournalCode,JournalLib,EcritureNum,EcritureDate,CompteNum,CompteLib,CompAuxNum,CompAuxLib,PieceRef,PieceDate,...,ValidDate,Montantdevise,Idevise,DateRglt,ModeRglt,NatOp,IdClient,Unnamed: 22,FILE_PATH,FILE_NAME
0,AN,A nouveaux,AN0000001,20180101,20500000,"BREVETS, LICENCES, LOGICIELS..",,,1,20180101,...,20190326,,EUR,,,,,NaN,/home/ftp/FEC-engine/inputs/000000000FEC201812...,000000000FEC20181231.txt


### Création de la base de donnée FEC

In [5]:
def create_datamodel(df_init, to_rename):
    # Init
    df = df_init.copy()
    
    # Map columns
    df = df.rename(columns=to_rename)
    
    # suppression des espaces colonne "COMPTE_NUM"
    df["COMPTE_NUM"] = df["COMPTE_NUM"].astype(str).str.strip()

    # Format des colonnes
    df = df.astype(
        {
            "FILE_NAME": str,
            "DATE": str,
            "COMPTE_NUM": str,
            "COMPTE_LIB": str,
            "ECRITURE_LIB": str,
            "DEBIT": float,
            "CREDIT": float,
        }
    )

    # Mise au format colonne date
    df["DATE"] = pd.to_datetime(df["DATE"])
    
    # Select col to keep
    to_select = [
        "FILE_NAME",
        "DATE",
        "COMPTE_NUM",
        "COMPTE_LIB",
        "ECRITURE_LIB",
        "DEBIT",
        "CREDIT",
    ]
    df = df[to_select]
    return df

df_output = create_datamodel(df_input, to_map)
print("✅ Row fetched:", len(df_output))
df_output.head(1)

✅ Row fetched: 4532


,FILE_NAME,DATE,COMPTE_NUM,COMPTE_LIB,ECRITURE_LIB,DEBIT,CREDIT
0,000000000FEC20181231.txt,2018-01-01,20500000,"BREVETS, LICENCES, LOGICIELS..",A Nouveau,4006.6,0.0


## Output

### Sauvegarde des fichiers en csv

In [6]:
def df_to_csv(df, output_folder, asset=False):
    # Create directory
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    # Create file path
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    file_name = f"{timestamp}_" + output_folder.split("/outputs/")[-1].replace("/", "_") + ".csv"
    
    # Sauvegarde en csv
    file_path = os.path.join(output_folder, file_name)
    df.to_csv(file_path, sep=";", decimal=",", index=False)
    print("✅ DataFrame saved in:", file_path)

    # Création du lien url
    if asset:
        naas_link = naas.asset.add(file_path)
        return naas_link

df_to_csv(df_output, output_folder_path)

✅ DataFrame saved in: /home/ftp/FEC-engine/outputs/FEC/BDD_FEC/20230524150131_FEC_BDD_FEC.csv
